## 1. Set up the environment
##### 1.1 Set up the SQL engine interface

In [ ]:
%reload_ext sql
%config SqlMagic.autocommit=False
%config SqlMagic.autolimit=0
%config SqlMagic.autopandas=True
%config SqlMagic.displaylimit=200

In [ ]:
%sql trino://localhost:9090/cuebiq/

##### 1.2 Install required additional packages

In [ ]:
!pip install keplergl -q -q
!pip install pyquadkey2 -q -q
!pip install h3 -q -q
!pip install pydeck -q -q

##### 1.3 Import pre-installed packages

In [ ]:
import os
from typing import List
import json
import copy
import itertools
import pandas as pd
import pydeck as pdk
from pyhive import trino

import geohash
from shapely.geometry import shape
from shapely.geometry import Polygon
from shapely.geometry import box
import geopandas as geopd

import seaborn as sns
import folium
from keplergl import KeplerGl

##### 1.4 User-defined functions

In [ ]:
from pyhive import trino

connection = trino.connect(
    host="localhost",
    port=9090
)

In [ ]:
schema_name = {'cda': 'cuebiq.paas_cda_pe_v3'}
output_schema_name = "ni_wb"

In [ ]:
#%sql show schemas from dedicated

In [ ]:
#%sql DROP SCHEMA IF EXISTS dedicated.ni_wb

In [ ]:
# %sql show schemas from dedicated

In [ ]:
# %sql create schema if not exists dedicated.$output_schema_name

In [ ]:
%sql show schemas from dedicated

In [ ]:
stop_table = f"{schema_name['cda']}.stop_uplevelled"

##### 1.5 Create Tables

In [ ]:
#Tables to be Created. Logic is create a table for each season. Seasons are defined by Equinox - Solstice. 

# Fall 2021
ni_wb_stops_table_F21 = f"dedicated.{output_schema_name}.ni_wb_stops_F21" # table to be created


In [ ]:
# Define the columns for the empty table. See Query in section 1.8
%sql create table if not exists $ni_wb_stops_table_F21 (cuebiq_id bigint, stop_zoned_datetime varchar, lat double, lng double, dwell_time_minutes double)

In [ ]:
%sql show tables from dedicated.$output_schema_name

##### 1.6 Set Date Range

In [ ]:
# Recommended method for selecting time period is via loop with list of days. See step 1.8 for application of dates_to_compute.
date_format = "%Y%m%d"
# Fall 2021
dates_to_compute = pd.date_range(start='2021-09-22', end='2021-12-20', freq='D').strftime(date_format)
dates_to_compute

##### 1.7 Set Polygon Filter

In [ ]:
# This polygon captures a larger area. A second 'Zone' filter is used in another step for selection of specific areas and is more detailed. 
# Can not use Geopandas in this environment. All spatial selections need to use WKT format. See ni_wkt is used in the Trino SQL statement below.
ni_wkt = "POLYGON ((-79.1535899775365 33.1787953098419,-79.189818350296 33.1808869589419,-79.2177765646679 33.232692686557,-79.2645789375597 33.2441079118145,-79.2810912803977 33.2536001463391,-79.287339439246 33.2760032156579,-79.2941333536436 33.2994327089762,-79.2951135331304 33.31442391375,-79.2925267462626 33.3271455410858,-79.2897907179897 33.3405484441312,-79.2812285991424 33.3474636609288,-79.2775672366869 33.3535382918033,-79.2741813153949 33.3597709872166,-79.272379137674 33.3616828115804,-79.2699272468766 33.3633865202096,-79.2685403558265 33.3651902659461,-79.267784542456 33.3663533726967,-79.2658822876999 33.3660975624434,-79.2631391967868 33.3658905955709,-79.2603417053701 33.365633305256,-79.257398522797 33.3653481006822,-79.2543382172084 33.3651226494656,-79.2488114484225 33.3645532388405,-79.2442606433493 33.3641726513259,-79.2380061662981 33.3639077135434,-79.2325678734866 33.363877734797,-79.2248522370016 33.3643246389541,-79.2219794648215 33.3644805264806,-79.2156860568072 33.3643431321676,-79.2067565696009 33.3644510933992,-79.2006763839836 33.3645854694008,-79.1936768183401 33.3645468386135,-79.1785593504818 33.3649736222825,-79.1701266023863 33.3653901668637,-79.169548995207 33.3658077150141,-79.1691096958964 33.3661378928832,-79.1684743592534 33.3660883382079,-79.1678811953241 33.3662640330461,-79.1684042944179 33.3671066967901,-79.1689630201797 33.3682852929736,-79.1687401996298 33.3694169949306,-79.1673585467443 33.3707107478342,-79.1644637518935 33.3717158836894,-79.1609689904341 33.3729533171509,-79.1584268686288 33.3734659941606,-79.1565914316933 33.3739981167791,-79.1548334172563 33.3747341163906,-79.1536338355467 33.3742770046899,-79.1532524356046 33.373381960197,-79.1532629066801 33.3721075466223,-79.1545948540822 33.3698447099512,-79.1558539435239 33.3673519792611,-79.1565510210607 33.3650809691719,-79.1561845391296 33.3646451217965,-79.1566048065776 33.3632099957948,-79.1569132127012 33.3620601934895,-79.1572005680017 33.3612806939062,-79.1576280133453 33.3595566956936,-79.1582128737101 33.3583113823549,-79.1587629136525 33.3573904470977,-79.1583745698891 33.3569349999668,-79.1578818149689 33.3565797450989,-79.1568803489977 33.3563136826673,-79.1513021874717 33.3567957858999,-79.150589650909 33.3600512807799,-79.1497006127499 33.3644285518789,-79.1493409590535 33.3674225211317,-79.1488683055858 33.3709939077628,-79.1481865985311 33.3742274026213,-79.1445634646652 33.3739918039801,-79.1398266036604 33.3737494680727,-79.1343409249391 33.3735765732316,-79.1476991782554 33.3030593400856,-79.160424849842 33.2300585257678,-79.1535899775365 33.1787953098419))"

##### 1.8 Run the Query

In [ ]:
for date in dates_to_compute:
    print(f"Executing date {date}")
    q = f"""
    INSERT INTO {ni_wb_stops_table_F21}
    SELECT
            cuebiq_id,
            stop_zoned_datetime,
            lat, 
            lng,
            dwell_time_minutes
    from {stop_table}
    where 
            country_code = 'US'
          and processing_date = {date}
          and st_contains(st_geometryfromtext(:ni_wkt)
          , st_point(lng,lat))
    """
    %sql $q

In [ ]:
# %sql SELECT COUNT(*) FROM {ni_wb_stops_table_F21}

##### 1.9 Cleanup

In [ ]:
# %sql DROP TABLE dedicated.ni_wb.ni_wb_stops_F21